### Notebook to process Coastal Erosion Typology
from geopackage to parquet

In [1]:
# Load software
import os
import pathlib
import sys
import json
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio

# Import custom functionality
from coclicodata.drive_config import p_drive

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

C:\Users\soest\AppData\Local\Temp\ipykernel_15216\3477376361.py:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [4]:
# Set path to geopackage
ds_dir = coclico_data_dir.joinpath('WP4','data','Erosion database')
ds_file = ds_dir.joinpath('CoCliCo_Erosion_database_240808.gpkg')

# Load data
gpkg = gpd.read_file(ds_file) # takes a while! Order of 15 minutes

In [5]:
gpkg

,Source,Country,Covered,SEG_ID,SEG_Length,Floodplain,L_FP,R_FP_1,R_FP_2,Onshore_structure,...,Harbour,Activ_geo,Barrier,Sed_type_1,Sed_type_2,Hist_Trend,Code_18,Recl_18,Notes,geometry
0,EUDEM,CY,No,CY000001,58,NaN,NaN,NaN,NaN,NaN,...,NaN,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((6400662.012 1600587.988, 640..."
1,EUDEM,CY,No,CY000002,86,NaN,NaN,NaN,NaN,NaN,...,NaN,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((6400689.502 1600639.893, 640..."
2,EUDEM,CY,No,CY000003,61,NaN,NaN,NaN,NaN,NaN,...,NaN,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((6400775.000 1600636.499, 640..."
3,EUDEM,CY,No,CY000004,83,NaN,NaN,NaN,NaN,NaN,...,NaN,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((6400835.352 1600638.892, 640..."
4,EUDEM,CY,No,CY000005,67,NaN,NaN,NaN,NaN,NaN,...,NaN,Not classified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTILINESTRING ((6400912.109 1600611.987, 640..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108815,EUDEM,ES,Yes,ES060573,63,Y,173346,0,0,Y,...,Y,Not active,NaN,No mobile sediment,NaN,Acc,123,8,NaN,"MULTILINESTRING ((3524462.115 1996763.011, 352..."
2108816,EUDEM,ES,Yes,ES060574,156,Y,173346,0,0,Y,...,Y,Not active,NaN,No mobile sediment,NaN,Acc,123,8,NaN,"MULTILINESTRING ((3524525.000 1996760.518, 352..."
2108817,EUDEM,ES,Yes,ES060575,289,Y,173346,0,0,Y,...,Y,Not active,NaN,No mobile sediment,NaN,Acc,123,8,NaN,"MULTILINESTRING ((3524680.867 1996766.696, 352..."
2108818,EUDEM,IT,Yes,IT067971,52,N,0,0,0,Y,...,Y,Not active,NaN,No mobile sediment,NaN,Acc,111,8,NaN,"MULTILINESTRING ((4745775.000 1931300.000, 474..."


In [10]:
# Write data as parquet file
parquet_file = str(ds_file).replace('.gpkg','.parquet')
gpkg.to_parquet(parquet_file)

In [9]:
# Test new parquet file
test_parquet = pd.read_parquet(parquet_file)
test_parquet

,Source,Country,Covered,SEG_ID,SEG_Length,Floodplain,L_FP,R_FP_1,R_FP_2,Onshore_structure,...,Harbour,Activ_geo,Barrier,Sed_type_1,Sed_type_2,Hist_Trend,Code_18,Recl_18,Notes,geometry
0,EUDEM,CY,No,CY000001,58,None,None,None,None,None,...,None,Not classified,None,None,None,None,None,None,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
1,EUDEM,CY,No,CY000002,86,None,None,None,None,None,...,None,Not classified,None,None,None,None,None,None,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
2,EUDEM,CY,No,CY000003,61,None,None,None,None,None,...,None,Not classified,None,None,None,None,None,None,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
3,EUDEM,CY,No,CY000004,83,None,None,None,None,None,...,None,Not classified,None,None,None,None,None,None,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
4,EUDEM,CY,No,CY000005,67,None,None,None,None,None,...,None,Not classified,None,None,None,None,None,None,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108815,EUDEM,ES,Yes,ES060573,63,Y,173346,0,0,Y,...,Y,Not active,None,No mobile sediment,None,Acc,123,8,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
2108816,EUDEM,ES,Yes,ES060574,156,Y,173346,0,0,Y,...,Y,Not active,None,No mobile sediment,None,Acc,123,8,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
2108817,EUDEM,ES,Yes,ES060575,289,Y,173346,0,0,Y,...,Y,Not active,None,No mobile sediment,None,Acc,123,8,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
2108818,EUDEM,IT,Yes,IT067971,52,N,0,0,0,Y,...,Y,Not active,None,No mobile sediment,None,Acc,111,8,NaN,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...
